# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [17]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [18]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that I delegate the task of continuously monitoring and analyzing market data for TSLA to the right team member who can utilize statistical modeling and machine learning effectively.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for TSLA using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for TSLA."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for TSLA to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/TSLA"} 

Tesla,

Final Answer: The stock price for Tesla, Inc. (TSLA) is currently at $394.74, with a slight decrease of 0.05%. The market cap for TSLA is $1.267T, and it operates in the Auto Manufacturers sector. TSLA has shown a YTD return of -5.43% and a 1-year return of 68.00%. The company designs, develops, manufactures, leases, and sells electric vehicles, as well as energy generation and storage systems. Additionally, TSLA offers various services such as automotive regulatory credits, vehicle insurance, and energy products to residential, commercial, and industrial customers.

> Finished chain.
 

The stock price for Tesla, Inc. (TSLA) is currently at $394.74, with a slight decrease of 0.05%. The market cap for TSLA is $1.267T, and it operates in the Auto Manufacturers sector. TSLA has shown a YTD return of -5.43% and a 1-year return of 68.00%. The company designs, develops, manufactures, leases, and sells electric vehicles, as well as energy generation and storage systems. Additionally, TSLA of

Final Answer: 
Based on the insights provided by the Data Analyst and the information gathered from Yahoo Finance on Tesla, Inc. (TSLA), I recommend developing trading strategies that align with the user-defined risk tolerance. It is essential to consider TSLA's stock price fluctuations, market cap, YTD return, and core business details to make informed decisions. By analyzing key performance metrics, financial highlights, and research analysis, we can create profitable and risk-averse trading strategies tailored to the user's risk tolerance level.

> Finished chain.
 

Based on the insights provided by the Data Analyst and the information gathered from Yahoo Finance on Tesla, Inc. (TSLA), I recommend developing trading strategies that align with the user-defined risk tolerance. It is essential to consider TSLA's stock price fluctuations, market cap, YTD return, and core business details to make informed decisions. By analyzing key performance metrics, financial highlights, and researc

I need to gather the necessary information from the Trading Strategy Developer and the Trade Advisor to determine the best execution methods for TSLA.

Action: Ask question to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "question": "What statistical modeling and machine learning techniques are you utilizing to identify trends and predict market movements for TSLA?", "context": "I need to understand the specific techniques being used to provide detailed execution plans."}

> Entering new CrewAgentExecutor chain...
I need to gather information on the statistical modeling and machine learning techniques used to identify trends and predict market movements for TSLA.

Action: Search the internet
Action Input: {"search_query": "statistical modeling and machine learning techniques for predicting market movements in TSLA"} 


Search results: Title: (PDF) Using Machine Learning Methods to Predict Tesla Stock
Link: https://www.researchgate.net/publication/376132920_Using_M

Thought:
I have gathered valuable insights on the statistical modeling and machine learning techniques used by the Trading Strategy Developer. Now, I need to inquire about the specific execution methods being recommended by the Trade Advisor to enhance the trading strategies for TSLA.

Action: Ask question to co-worker
Action Input: {"coworker": "Trade Advisor", "question": "What specific execution methods are you recommending to enhance the trading strategies for TSLA, considering its volatility and liquidity?", "context": "I need to understand the recommended execution methods to provide detailed execution plans."}

> Entering new CrewAgentExecutor chain...
I need to gather information on the specific execution methods recommended for enhancing trading strategies for TSLA, taking into account its volatility and liquidity.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/trading-tesla-tsla-4771283"} 

Page Not Found
Skip to content
 Investing
 

Final Answer: Based on the information gathered from the website content on volatility, it is essential to consider the statistical measure of dispersion of returns for TSLA when recommending specific execution methods for trading strategies. Understanding TSLA's volatility and how it relates to risk and uncertainty will be crucial in optimizing trade execution strategies for this stock.

> Finished chain.
 

Based on the information gathered from the website content on volatility, it is essential to consider the statistical measure of dispersion of returns for TSLA when recommending specific execution methods for trading strategies. Understanding TSLA's volatility and how it relates to risk and uncertainty will be crucial in optimizing trade execution strategies for this stock.

Final Answer:
Detailed execution plans suggesting how and when to execute trades for TSLA should involve leveraging statistical modeling and machine learning techniques to identify trends and predict market mo

Final Answer:
Risk Analysis Report for Trading Strategies and Execution Plans for TSLA:

Introduction:
Tesla Inc. (TSLA) is a leading electric vehicle and clean energy company that has experienced significant growth and volatility in the stock market. The following risk analysis report aims to identify potential risks associated with the proposed trading strategies and execution plans for TSLA and provide recommendations for risk mitigation.

Finance and Corporate Risks:
One of the key risks associated with trading TSLA stock is finance and corporate risks. Tesla's financial performance, cash flow, debt levels, and ability to meet its financial obligations can impact the stock price. Additionally, corporate governance issues, management changes, and strategic decisions can introduce uncertainties for investors.

Production Risks:
Tesla's production processes, supply chain disruptions, manufacturing challenges, and delays in product deliveries can pose risks to the company's stock perfo

- Display the final result as Markdown.

In [19]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Trading Strategies and Execution Plans for TSLA:

Introduction:
Tesla Inc. (TSLA) is a leading electric vehicle and clean energy company that has experienced significant growth and volatility in the stock market. The following risk analysis report aims to identify potential risks associated with the proposed trading strategies and execution plans for TSLA and provide recommendations for risk mitigation.

Finance and Corporate Risks:
One of the key risks associated with trading TSLA stock is finance and corporate risks. Tesla's financial performance, cash flow, debt levels, and ability to meet its financial obligations can impact the stock price. Additionally, corporate governance issues, management changes, and strategic decisions can introduce uncertainties for investors.

Production Risks:
Tesla's production processes, supply chain disruptions, manufacturing challenges, and delays in product deliveries can pose risks to the company's stock performance. Any issues related to production efficiency, quality control, or capacity constraints may impact investor confidence and stock valuation.

Legal and Regulatory Risks:
Legal and regulatory risks are critical considerations when trading TSLA stock. Tesla operates in a highly regulated industry with evolving standards, environmental regulations, safety requirements, and intellectual property concerns. Legal disputes, regulatory investigations, or compliance issues can have adverse effects on Tesla's stock price.

Technology and Innovation Risks:
As a technology-driven company, Tesla faces risks related to technological advancements, product innovation, competitive pressures, and intellectual property protection. Changes in consumer preferences, technological disruptions, or failure to maintain a competitive edge in the market can affect Tesla's stock performance.

Ability to Sell Risks:
The ability to sell Tesla's products, maintain market share, expand into new markets, and generate sustainable revenue growth are essential factors for stock valuation. Market demand, pricing strategies, brand reputation, and customer satisfaction levels influence Tesla's ability to sell its products and services.

Macroeconomic and Political Risks:
Macroeconomic factors such as interest rates, inflation, economic growth, trade policies, and geopolitical events can impact Tesla's stock price. Political uncertainties, government regulations, trade tensions, and global market conditions create risks for Tesla's operations and financial performance.

Execution Risk:
The effective execution of trading strategies and implementation of trade plans for TSLA involve execution risk. Market timing, order placement, trade execution speed, liquidity constraints, and unexpected market events can affect the outcome of trades. Traders must carefully manage execution risk to optimize trading outcomes.

High Valuation and Market Sentiment Risks:
The high valuation of Tesla stock relative to earnings, market multiples, and industry peers poses risks for investors. Market sentiment, speculative trading, short-term price fluctuations, and valuation bubbles can lead to price volatility and investment losses. Traders should consider valuation metrics and market sentiment indicators when making trading decisions.

Impact of Low-Cost Models and Competitive Pressures:
Tesla's expansion into low-cost models, mass-market vehicles, and competitive pricing strategies introduce risks related to profitability, sales volumes, market share, and brand differentiation. Competitive pressures from traditional automakers, electric vehicle rivals, and new market entrants can impact Tesla's market position and stock performance.

Battery Supply Chain and Technological Challenges:
Tesla's reliance on battery technology, supply chain logistics, raw material sourcing, and production scalability create risks for the company's operations and growth prospects. Technological challenges, battery shortages, cost pressures, and regulatory changes in the energy storage sector can affect Tesla's stock price and business sustainability.

Recommendations for Risk Mitigation:
Based on the identified risks and challenges associated with trading TSLA stock, the following recommendations are proposed for risk mitigation:

1. Conduct comprehensive due diligence on Tesla's financial health, corporate governance, and strategic direction.
2. Monitor production updates, supply chain resilience, and product quality assurance measures to mitigate production risks.
3. Stay informed about legal and regulatory developments, compliance requirements, and industry standards impacting Tesla.
4. Track technological advancements, innovation trends, and competitive landscape changes to assess technology risks.
5. Evaluate market demand, pricing dynamics, customer feedback, and brand positioning to manage selling risks effectively.
6. Monitor macroeconomic indicators, political developments, and global market trends influencing Tesla's stock performance.
7. Implement risk management strategies for trade execution, order handling, liquidity management, and trade monitoring.
8. Analyze valuation metrics, market sentiment indicators, and investment risks related to Tesla's stock price.
9. Assess the impact of low-cost models, competitive pressures, and market disruptions on Tesla's business strategy.
10. Address battery supply chain risks, technological challenges, and energy storage innovations to enhance Tesla's operational resilience.

Conclusion:
In conclusion, trading TSLA stock involves inherent risks and uncertainties that require careful evaluation, risk analysis, and proactive risk management strategies. By identifying potential risks, understanding market dynamics, and implementing risk mitigation measures, traders can optimize trading strategies and execution plans for TSLA based on current market conditions and pricing dynamics. It is essential to stay informed, monitor risk factors, and adapt trading strategies to mitigate risks effectively and achieve trading objectives in the dynamic stock market environment.